## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-23-49-12 +0000,nyt,Trump Says a Deal With Harvard Is Close,https://www.nytimes.com/2025/09/30/us/trump-ha...
1,2025-09-30-23-48-27 +0000,nyt,Hegseth Lectures Military Brass on ‘Woke’ Lead...,https://www.nytimes.com/2025/09/30/us/politics...
2,2025-09-30-23-46-18 +0000,nyt,Trump and Hegseth Recount Familiar Partisan Co...,https://www.nytimes.com/2025/09/30/us/politics...
3,2025-09-30-23-40-11 +0000,nypost,Skeletal remains found near NYC strip mall bel...,https://nypost.com/2025/09/30/us-news/skeletal...
4,2025-09-30-23-35-53 +0000,nyt,Schumer and Thune Spar as Government Shutdown ...,https://www.nytimes.com/2025/09/30/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,58
32,shutdown,24
31,government,21
1,deal,18
106,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
306,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,159
9,2025-09-30-23-28-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...,142
91,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...,130
291,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,125
93,2025-09-30-20-20-33 +0000,nypost,Trump says Harvard will run new trade schools ...,https://nypost.com/2025/09/30/us-news/trump-an...,122


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
306,159,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
9,65,2025-09-30-23-28-00 +0000,wsj,The government was on course to shut down earl...,https://www.wsj.com/politics/policy/government...
91,61,2025-09-30-20-27-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
152,55,2025-09-30-17-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
224,54,2025-09-30-13-35-39 +0000,nyt,"With New U.S. Proposal to End Gaza War, a Rare...",https://www.nytimes.com/2025/09/30/world/middl...
68,41,2025-09-30-21-27-55 +0000,nypost,Eric Adams is so anti-Zohran Mamdani he’s warm...,https://nypost.com/2025/09/30/us-news/eric-ada...
179,37,2025-09-30-15-50-00 +0000,bbc,Trump peace plan 'ignores interests of Palesti...,https://www.bbc.com/news/articles/cx2j97jldkmo...
19,34,2025-09-30-23-10-05 +0000,nypost,Trump’s energy secretary rails against NY gree...,https://nypost.com/2025/09/30/us-news/trumps-e...
47,32,2025-09-30-22-04-22 +0000,nypost,Ford CEO warns there’s not enough blue-collar ...,https://nypost.com/2025/09/30/business/ford-ce...
225,31,2025-09-30-13-33-28 +0000,nypost,Wild video shows drunk passenger being yanked ...,https://nypost.com/2025/09/30/world-news/drunk...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
